In [24]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, f1_score

In [25]:
train_data = pd.read_csv('../data_new/train.csv')
test = pd.read_csv('../data_new/test.csv')

C:\Users\ADE17\AppData\Local\Temp\ipykernel_12648\2619112913.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('../data_new/train.csv')


In [26]:
target = train_data.iloc[:,-29:]
target_cols = train_data.iloc[:,-29:].columns

In [28]:
train_data.Blood_Month_sample = train_data.Blood_Month_sample.fillna(train_data.Blood_Month_sample.mean())
train_data.Blood_Month_sample = train_data.Blood_Month_sample.astype('int64')
# train_data.General_cofactors = train_data.General_cofactors.fillna(train_data.General_cofactors.mean())
# train_data.General_cofactors = train_data.General_cofactors.astype('int64')

In [32]:
# train_data.General_ocofactors = train_data.General_cofactors.fillna(train_data.General_cofactors.mean())
# train_data.General_cfactors = train_data.General_cofactors.astype('int64')
train_data.General_cofactors.value_counts()

0                 2085
10                 652
1                   95
5                   43
6                   32
7                   21
4                   14
3                    7
6, 7                 5
2                    4
1, 3                 3
1, 2                 2
6,7                  2
4, 7                 2
1,3                  2
1.5                  1
6, 8                 1
5, 6, 7, 10          1
2, 12                1
4, 6                 1
7, 10                1
6, 7, 8, 10          1
4, 5, 6, 7, 10       1
4, 5                 1
1, 6                 1
11                   1
4, 6, 7, 11          1
6, 7, 10             1
4,6                  1
1, 6, 7, 10          1
12                   1
6, 7                 1
5, 6, 7              1
4, 6, 7              1
5, 7                 1
Name: General_cofactors, dtype: int64

In [4]:
train = train_data.drop(target, axis=1)

In [5]:
def multival_onehot_enc(df, col_names):
    for col_name in col_names:
        print(col_name)
        df[col_name] = df[col_name].str.replace('.', ',')

        split_values = df[col_name].str.split(',').explode()
        unique_values = split_values.unique()

        for i, value in enumerate(unique_values):
            df[f"{col_name}_{i}"] = 0
            df[f"{col_name}_{i}"] = df[f"{col_name}_{i}"].astype(int)

        for index, row in df.iterrows():
            values = str(row[col_name]).split(',')
            for value in values:
                for i, unique_value in enumerate(unique_values):
                    if value.strip() == unique_value:
                        df.at[index, f"{col_name}_{i}"] = 1
                        break
        df.drop(col_name, axis=1, inplace= True)
    
    return df


In [6]:
cols = ['Treatment_of_athsma', 'General_cofactors',
        'Treatment_of_atopic_dematitis', 'Age_of_onsets']
cols2 = ['French_Region', 'Chip_Type', 
        'Blood_Month_sample', 'Skin_Symptoms',
        'Treatment_of_rhinitis']
train.Treatment_of_rhinitis = train.Treatment_of_rhinitis.astype(str).str.split(pat='.', expand=True)[0]
train = pd.get_dummies(train, columns=cols2)
new_train = multival_onehot_enc(train, cols)

Treatment_of_athsma


C:\Users\ADE17\AppData\Local\Temp\ipykernel_12648\3692048208.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col_name] = df[col_name].str.replace('.', ',')


General_cofactors


C:\Users\ADE17\AppData\Local\Temp\ipykernel_12648\3692048208.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col_name] = df[col_name].str.replace('.', ',')


Treatment_of_atopic_dematitis


C:\Users\ADE17\AppData\Local\Temp\ipykernel_12648\3692048208.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col_name] = df[col_name].str.replace('.', ',')


Age_of_onsets


C:\Users\ADE17\AppData\Local\Temp\ipykernel_12648\3692048208.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col_name] = df[col_name].str.replace('.', ',')


In [130]:
# new_train

In [7]:
df_num = new_train.select_dtypes(include=object)


In [8]:
new_train = new_train.drop(df_num, axis=1)

In [9]:
cols_test = ['Treatment_of_athsma', 'General_cofactors',
        'Treatment_of_atopic_dematitis']
cols2_test = ['French_Region', 'Chip_Type', 
        'Blood_Month_sample', 'Skin_Symptoms',
        'Treatment_of_rhinitis', 'Age_of_onsets']
test.Treatment_of_rhinitis = test.Treatment_of_rhinitis.astype(str).str.split(pat='.', expand=True)[0]
test = pd.get_dummies(test, columns=cols2_test)
new_test = multival_onehot_enc(test, cols_test)

Treatment_of_athsma
General_cofactors
Treatment_of_atopic_dematitis


C:\Users\ADE17\AppData\Local\Temp\ipykernel_12648\3692048208.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col_name] = df[col_name].str.replace('.', ',')
C:\Users\ADE17\AppData\Local\Temp\ipykernel_12648\3692048208.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col_name] = df[col_name].str.replace('.', ',')
C:\Users\ADE17\AppData\Local\Temp\ipykernel_12648\3692048208.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col_name] = df[col_name].str.replace('.', ',')


In [10]:
df_num_test = new_test.select_dtypes(include=object)


In [11]:
new_test = new_test.drop(df_num_test, axis=1)

In [12]:
new_test = new_test.drop('trustii_id', axis=1)

In [13]:
new_test.shape

(586, 401)

In [14]:
new_train.shape

(2989, 412)

In [15]:
df1 = new_train
df2 = new_test
columns_df1 = set(df1.columns)
columns_df2 = set(df2.columns)

different_columns = list(columns_df1 - columns_df2)
different_columns.extend(list(columns_df2 - columns_df1))

In [16]:
different_columns

['Blood_Month_sample_9.0',
 'General_cofactors_16',
 'Blood_Month_sample_12.0',
 'General_cofactors_18',
 'French_Region_regionO',
 'French_Region_regionN',
 'General_cofactors_20',
 'Blood_Month_sample_6.0',
 'Blood_Month_sample_7.0',
 'Blood_Month_sample_1.0',
 'Blood_Month_sample_2.0',
 'Age_of_onsets_8',
 'Treatment_of_athsma_10',
 'Blood_Month_sample_4.0',
 'Blood_Month_sample_3.0',
 'Blood_Month_sample_5.0',
 'Blood_Month_sample_8.0',
 'Blood_Month_sample_10.0',
 'General_cofactors_19',
 'Blood_Month_sample_11.0',
 'Age_of_onsets_7',
 'General_cofactors_17',
 'Treatment_of_atopic_dematitis_8',
 'Blood_Month_sample_10',
 'Blood_Month_sample_9',
 'Blood_Month_sample_1',
 'Blood_Month_sample_4',
 'Blood_Month_sample_7',
 'Blood_Month_sample_2',
 'Blood_Month_sample_6',
 'Blood_Month_sample_8',
 'Blood_Month_sample_5',
 'Blood_Month_sample_12',
 'Blood_Month_sample_3',
 'Blood_Month_sample_11']

In [132]:
test.Blood_Month_sample.value_counts()

6     68
3     64
1     60
9     57
2     49
5     49
12    49
7     47
4     40
11    40
10    38
8     25
Name: Blood_Month_sample, dtype: int64

In [ ]:
# train_data.Blood_Month_sample = train_data.Blood_Month_sample.fillna(train_data.Blood_Month_sample.mean())
# train_data.Blood_Month_sample = train_data.Blood_Month_sample.astype('int64')

In [21]:
# train_data.General_cofactors = train_data.General_cofactors.fillna(train_data.General_cofactors.mean())
# train_data.General_cofactors = train_data.General_cofactors.astype('int64')

AttributeError: 'Series' object has no attribute 'General_cofactors'

In [23]:
train_data

0        7
1        5
2        6
3        9
4       12
        ..
2984     3
2985     1
2986    12
2987     4
2988     7
Name: Blood_Month_sample, Length: 2989, dtype: int64